# 🚀 Ailo Network - Colab Miner (AILO-1B)

**Mine AiloCoin with Google Colab's free T4 GPU!**

1. Enable GPU: Runtime > Change runtime type > T4
2. Enter wallet in cell 2
3. Runtime > Run all

In [ ]:
#@title ⚙️ 1. Setup
!pip install torch --index-url https://download.pytorch.org/whl/cu118 -q
!pip install aiohttp requests -q

import gc
gc.collect()

import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f} GB")
else:
    print("❌ Enable GPU: Runtime > Change runtime type > T4")

In [ ]:
#@title 🔑 2. Wallet
WALLET = ""  #@param {type:"string"}
if len(WALLET) < 40:
    print("❌ Enter wallet from https://ailo.site/wallet.html")
else:
    print(f"✅ Wallet: {WALLET[:12]}...{WALLET[-8:]}")

In [ ]:
#@title 🧠 3. AILO-1B Model

import torch
import torch.nn as nn
import numpy as np
import requests
import aiohttp
import asyncio
import time
import base64
import gzip
import gc

API = "https://ailo.site/api"
VER = "1.3.1-colab"
BATCH = 1
SEQ = 32
ACCUM = 16
SUBMIT_SEC = 300
HEARTBEAT_SEC = 10  # Send heartbeat every 10 seconds

class AILO1B(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(50257, 1600)
        self.pos = nn.Parameter(torch.zeros(1, 512, 1600))
        layer = nn.TransformerEncoderLayer(d_model=1600, nhead=25, dim_feedforward=6400, dropout=0.1, batch_first=True)
        self.tf = nn.TransformerEncoder(layer, num_layers=24)
        self.out = nn.Linear(1600, 50257)

    def forward(self, x):
        x = self.emb(x) * 40.0 + self.pos[:, :x.size(1), :]
        return self.out(self.tf(x))

class Trainer:
    def __init__(self):
        self.dev = torch.device('cuda')
        self.model = None
        self.opt = None
        self.crit = nn.CrossEntropyLoss()
        self.steps = 0
        self.acc = 0

    def init(self):
        print("🧠 Loading AILO-1B...")
        gc.collect()
        torch.cuda.empty_cache()
        
        with torch.cuda.amp.autocast():
            self.model = AILO1B().half().to(self.dev)
        
        p = sum(x.numel() for x in self.model.parameters())
        print(f"   📐 {p:,} params ({p/1e9:.2f}B)")
        print(f"   💾 GPU: {torch.cuda.memory_allocated()/1024**3:.1f} GB")
        
        self.opt = torch.optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        print("   ✅ Ready!")

    def batch(self, w):
        try:
            r = requests.get(f"https://ailo.site/api/cuda/training-data", params={'batchSize': BATCH, 'wallet': w}, timeout=10)
            if r.ok:
                texts = r.json().get('articles', [])
                if texts:
                    bx, by = [], []
                    for t in texts:
                        tk = [ord(c) % 50257 for c in t[:SEQ+1]]
                        tk += [0] * (SEQ+1 - len(tk))
                        bx.append(tk[:SEQ])
                        by.append(tk[1:SEQ+1])
                    return torch.tensor(bx), torch.tensor(by)
        except: pass
        return torch.randint(0, 50257, (BATCH, SEQ)), torch.randint(0, 50257, (BATCH, SEQ))

    def step(self, x, y):
        self.model.train()
        x, y = x.to(self.dev), y.to(self.dev)
        
        with torch.cuda.amp.autocast():
            o = self.model(x)
            loss = self.crit(o.view(-1, 50257), y.view(-1)) / ACCUM
        
        loss.backward()
        self.acc += 1
        
        if self.acc >= ACCUM:
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.opt.step()
            self.opt.zero_grad(set_to_none=True)
            self.acc = 0
        
        self.steps += 1
        if self.steps % 100 == 0:
            torch.cuda.empty_cache()
        
        return loss.item() * ACCUM

    def grads(self):
        g = [p.grad.cpu().float().flatten() for p in self.model.parameters() if p.grad is not None]
        if not g: return None
        return base64.b64encode(gzip.compress(torch.cat(g).half().numpy().tobytes(), 4)).decode()

print("✅ AILO-1B defined")

In [ ]:
#@title 🚀 4. Mine!

async def submit(w, g, s, l):
    try:
        async with aiohttp.ClientSession() as sess:
            async with sess.post(f"{API}/cuda/submit", json={'wallet': w, 'gradients': g, 'epoch': s, 'loss': l, 'gpu': 'T4-Colab'}) as r:
                if r.status == 200:
                    return (await r.json()).get('reward', 0)
    except: pass
    return 0

async def heartbeat(w, hashrate, loss):
    """Send heartbeat to keep miner visible in dashboard."""
    try:
        async with aiohttp.ClientSession() as s:
            # Update CUDA client registration (keeps miner in cudaClients map)
            await s.post(f"{API}/cuda/register", json={
                'wallet': w,
                'clientVersion': VER,
                'deviceInfo': {
                    'gpu_name': 'Tesla T4 (Colab)',
                    'vram_gb': 15,
                    'hashrate': hashrate
                }
            })
            # Also send ping for activeNodes (dashboard visualizer)
            await s.post(f"{API}/ping", json={
                'wallet': w,
                'status': 'training',
                'hashrate': f"{hashrate:.0f} Tok/s"
            })
    except:
        pass

async def run():
    if len(WALLET) < 40:
        print("❌ Set wallet!")
        return

    print("="*40)
    print(f"  AILO-1B Colab Miner v{VER}")
    print("="*40)

    # Initial registration
    await heartbeat(WALLET, 0, 0)
    print("📝 Registered with server")

    t = Trainer()
    t.init()

    print("\n⛏️ MINING!")
    print("📊 Check: https://ailo.site/dashboard.html\n")

    rewards = 0.0
    last_submit = time.time()
    last_heartbeat = time.time()
    current_loss = 0.0
    current_tps = 0.0

    try:
        while True:
            t0 = time.time()
            x, y = t.batch(WALLET)
            current_loss = t.step(x, y)
            current_tps = (BATCH * SEQ) / max(0.001, time.time() - t0)

            # Send heartbeat every HEARTBEAT_SEC seconds
            if time.time() - last_heartbeat >= HEARTBEAT_SEC:
                await heartbeat(WALLET, current_tps, current_loss)
                last_heartbeat = time.time()

            if t.steps % 25 == 0:
                print(f"Step {t.steps:4d} | Loss: {current_loss:.4f} | {current_tps:.0f} tok/s | GPU: {torch.cuda.memory_allocated()/1024**3:.1f}GB")

            if time.time() - last_submit >= SUBMIT_SEC:
                print("\n📤 Submitting gradients...")
                g = t.grads()
                if g:
                    r = await submit(WALLET, g, t.steps, current_loss)
                    rewards += r
                    print(f"💰 +{r:.4f} ALC (Total: {rewards:.4f})")
                print("📊 Dashboard updated!\n")
                last_submit = time.time()

    except KeyboardInterrupt:
        print(f"\n⏹️ Done. Total: {rewards:.4f} ALC")

await run()